In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[13]: [FileInfo(path='dbfs:/FileStore/tables/Dataframes_Assignment__1_.ipynb', name='Dataframes_Assignment__1_.ipynb', size=34543, modificationTime=1652224278000),
 FileInfo(path='dbfs:/FileStore/tables/New_Text_Document.txt', name='New_Text_Document.txt', size=2637452, modificationTime=1646128769000),
 FileInfo(path='dbfs:/FileStore/tables/W6S1Demo_mapreduce.ipynb', name='W6S1Demo_mapreduce.ipynb', size=12935, modificationTime=1647192093000),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts-1.zip', name='accounts-1.zip', size=5297592, modificationTime=1647963102000),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1645626546000),
 FileInfo(path='dbfs:/FileStore/tables/character_counting-1.ipynb', name='character_counting-1.ipynb', size=25386, modificationTime=1645912625000),
 FileInfo(path='dbfs:/FileStore/tables/character_counti

In [0]:
#filepath stored in filepath variable
filepath = "/FileStore/tables/clinicaltrial_2021_csv.gz"

In [0]:
# file copied from the filepath to tmp
dbutils.fs.cp(filepath,"file:/tmp/")

Out[15]: True

In [0]:
%sh
gzip -d /tmp/ /tmp/clinicaltrial_2021_csv.gz

gzip: /tmp/ is a directory -- ignored


In [0]:
# file moved to specific directory
dbutils.fs.mv("file:/tmp/clinicaltrial_2021_csv","/FileStore/tables/")

Out[17]: True

In [0]:
# prepared to the required csv format
dbutils.fs.mv("/FileStore/tables/clinicaltrial_2021_csv","/FileStore/tables/clinicaltrial_2021.csv")

Out[18]: True

In [0]:
# read data to PySpark with delimiter pipe.
clinicaltrial_2021 = spark.read.option("header","true").csv("/FileStore/tables/clinicaltrial_2021.csv", sep="|").rdd

In [0]:
# Question 1
clinicaltrial_2021.distinct().count()

Out[20]: 387261

In [0]:
# Question 2
clinicalrdd = clinicaltrial_2021.map(lambda e: (e[5],1)).reduceByKey(lambda e1,e2: e1+e2)
clinicalrdd.sortBy(lambda e : e[1],False).collect()

Out[21]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
# Question 3
clinicaltrial_2021_flat = clinicaltrial_2021.filter(lambda e:e[7] is not None).flatMap(lambda e:e[7].split(","))
clinicaltrial_2021_sort = clinicaltrial_2021_flat.map(lambda e: (e,1)).reduceByKey(lambda e1,e2: e1+e2)
clinicaltrial_2021_sort.sortBy(lambda e : e[1],False).take(5)

Out[22]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
# Question 4
mesh = spark.read.option("header","true").csv("/FileStore/tables/mesh.csv").rdd
mesh_rdd = mesh.map(lambda e:(e[0],e[1].split('.')[0]))
clinicaltrial_2021_join = clinicaltrial_2021_flat.map(lambda e: (e,1)).join(mesh_rdd)
clinicaltrial_2021_join.map(lambda e:(e[1][1],e[1][0])).reduceByKey(lambda e1,e2: e1+e2).sortBy(lambda e: e[1],False).take(10)

Out[23]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310),
 ('C06', 85646),
 ('C08', 70720),
 ('C13', 42599),
 ('C18', 41276),
 ('C12', 40161)]

In [0]:
# Question 5
pharma = spark.read.option("header","true").csv("/FileStore/tables/pharma.csv").rdd
parentrdd = pharma.map(lambda e:(e[1],1))
sponsorrdd = clinicaltrial_2021.map(lambda e:(e[1],1))
clinicaltrial_2021_join = sponsorrdd.leftOuterJoin(parentrdd).filter(lambda e:e[1][1]!=1)
clinicaltrial_2021_join.map(lambda e: (e[0],e[1][0])).reduceByKey(lambda e1,e2: e1+e2).sortBy(lambda e: e[1],False).take(10)

Out[24]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
#Question 6 
clinical_filter = clinicaltrial_2021.filter(lambda e:e[2] == "Completed").filter(lambda e:e[4] is not None).filter(lambda e:e[4].split(' ')[1] == "2021")
clinical_filter.map(lambda e:(e[4].split(' ')[0],1)).reduceByKey(lambda e1,e2: e1+e2).sortBy(lambda e: e[1],False).collect()

Out[25]: [('Mar', 1227),
 ('Jan', 1131),
 ('Jun', 1094),
 ('May', 984),
 ('Apr', 967),
 ('Feb', 934),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]